<a href="https://colab.research.google.com/github/Conan486/Raivath_coding_challenge/blob/main/hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install pymongo

In [24]:
#import packages
import time
from copy import deepcopy
import json
import pymongo
import os
import datetime
import pandas
import math

In [25]:
#get the username and and password information - You need to run it one time at start
from getpass import getpass

USER = "ubuntu"
PASS = "Greenguy678"
HOST = "13.220.136.235"
PORT = "27017"

In [26]:
#set up mongo connection
mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(USER,
                                                         PASS,
                                                         HOST,
                                                         PORT,
                                                         "lahman")

client = pymongo.MongoClient(mongo_url)

In [27]:
# Testing connection!
db = client['lahman']
#now print all collection names
print("collection names: {}".format(db.list_collection_names()))
reading = db['Batting'].find_one()
#print one batting documents
reading["_id"] = str(reading["_id"])
print(reading)

collection names: ['Managers', 'AwardsPlayers', 'Master', 'HallOfFame', 'Fielding', 'AwardsSharePlayers', 'Appearances', 'Teams', 'AllstarFull', 'AwardsShareManagers', 'Batting', 'Schools', 'ManagersHalf', 'Salaries', 'AwardsManagers', 'TeamsHalf', 'TeamsFranchises', 'Pitching', 'FieldingOF', 'SchoolsPlayers', 'SeriesPost', 'PitchingPost', 'FieldingPost']
{'_id': '698a578f62a5cb91ff4dd085', 'playerID': 'aardsda01', 'yearID': 2004, 'stint': 1, 'teamID': 'SFN', 'lgID': 'NL', 'G': 11, 'G_batting': 11, 'AB': 0, 'R': 0, 'H': 0, '2B': 0, '3B': 0, 'HR': 0, 'RBI': 0, 'SB': 0, 'CS': 0, 'BB': 0, 'SO': 0, 'IBB': 0, 'HBP': 0, 'SH': 0, 'SF': 0, 'GIDP': 0, 'G_old': 11}


## Part 1 Queries

1. The number of all stars in allstarfull.
2. The most home runs in a season by a single player (using the batting table).
3. The playerid of the player with the most home runs in a season.
4. The number of leagues in the batting table.
5. Barry Bond's average batting average (playerid = 'bondsba01') where batting average is hits / at-bats. Note you will nead to cast hits to get a decimal: cast(h as real)
6. The teamid with the fewest hits in the year 2000 (ie., yearid = '2000'). Return both the teamid, and the number of hits. Note you can use ORDER BY column and LIMIT 1.
7. The teamid in the year 2000 (i.e., yearid = '2000') with the highest average batting average. Return the teamid and the average. To prevent divsion by 0, limit at-bats > 0.
8. The number of all stars the giants (teamid = 'SFN') had in 2000.
9. The yearid which the giants had the most all stars.
10. The average salary in year 2000.
11. The number of positions (e.g., catchers, pitchers) that have average salaries greather than 2000000 in yearid 2000. You will need to join fielding with salaries.
12. The number of errors Barry Bonds had in 2000.
13. The average salary of all stars in 2000.
14. The average salary of non-all stars in 2000.

In [42]:
def q1():
  # Returns number of documents (rows) in the all stars full table by simply counting them
 return db['AllstarFull'].count_documents({})


def q2():
  # Return 0 if the table is empty
  if db["Batting"].count_documents({}) == 0:
    return 0

  # Return the hr value of the first document after sorting the table by descending hr values and filtering out all empty values
  return db["Batting"].find_one({"HR": {"$ne": ""}}, sort=[("HR", -1)])["HR"]


def q3():
  # Return null if the table is empty
  if db["Batting"].count_documents({}) == 0:
    return None

  # Return the player id value of the first document after sorting the table by descending hr values and filtering out all empty values
  return db["Batting"].find_one({"HR": {"$ne": ""}}, sort=[("HR", -1)])["playerID"]


def q4():
  # Return the length of the distinct lgID values list
  return len(db["Batting"].distinct("lgID"))



def q5():
  # Aggregation pipeline to calculate average batting average
   res = list(db["Batting"].aggregate([
        # Select documents about Barry Bonds
        {"$match": {"playerID": "bondsba01", "AB": {"$gt" : 0}}},

        # Calculate a new column holding batting average
        {"$project": {"bat_avg": {"$divide": ["$H", "$AB"]}}},

        # Average the batting average
        {"$group": {"_id": None,
                    "avg_bat_avg" : {"$avg": "$bat_avg"}}}

    ]))
   return res[0]["avg_bat_avg"]


def q6():
  # Sorts the teams in year 2000 by ascending hits and picks the first document
  res = db['Teams'].find_one({'yearID': 2000}, sort=[('H', 1)])
  return (res['teamID'], res['H'])


def q7():
  res = list(db['Batting'].aggregate([
        # Select documents with year = 2000 and AB greater than 0 only
        {'$match': {'yearID': 2000, 'AB': {'$gt': 0}}},

        # Calculates the batting average and retain the teamID column to group by later
        {'$project': {'teamID': 1,
                      'bat_avg': {'$divide': ['$H', '$AB']}}},

        # Group by team and calculate the average of batting average
        {'$group': {'_id': '$teamID',
                    'avg_bat_avg': {'$avg': '$bat_avg'}}},

        # Sort by avgerage batting average of each time in descending order
        {'$sort': {'avg_bat_avg': -1}},

        # Pick the top document
        {'$limit': 1}
    ]))

  return (res[0]['_id'], res[0]['avg_bat_avg'])


def q8():
  # Count all documents after filtering team and year
  return db["AllstarFull"].count_documents({"teamID": "SFN", "yearID": 2000})



def q9():
  # Aggregation pipeline to find the year when Giants had the most all stars
  res = list(db['AllstarFull'].aggregate([
      # Filter for Giants (teamID: "SFN")
      {'$match': {'teamID': 'SFN'}},

      # Group by year and count all stars
      {'$group': {'_id': '$yearID',
                  'count': {'$sum': 1}}},

      # Sort by count in each year in descending order
      {'$sort': {'count': -1}},

      # Pick the top document
      {'$limit': 1}
  ]))

  return res[0]['_id']



def q10():
  res = list(db['Salaries'].aggregate([
      # Filter documents of year 2000
      {"$match": {"yearID": 2000}},

      # Calculate average salary
      {"$group": {"_id": None, "avg_salary": {"$avg": "$salary"}}}
  ]))

  return res[0]['avg_salary']


def q11():
  res = list(db['Fielding'].aggregate([
        # Filter for year 2000
        {'$match': {'yearID': 2000}},

        # Join with Salaries collection on playerID
        {'$lookup': {'from': 'Salaries',
                      'localField': 'playerID',
                      'foreignField': 'playerID',
                      'as': 'salary_info'}},

        # Unwind (flatten) the salary_info array
        {'$unwind': '$salary_info'},

        # Filter only salaries from year 2000
        {'$match': {'salary_info.yearID': 2000}},

        # Group by position and calculate average salary
        {'$group': {'_id': '$POS',
                    'avg_salary': {'$avg': '$salary_info.salary'}}},

        # Filter groups where average salary > 2000000
        {'$match': {'avg_salary': {'$gt': 2000000}}},

        # Count the number of positions
        {'$count': 'num_pos'}
    ]))

  return res[0]['num_pos']



def q12():
  res = list(db['Fielding'].aggregate([
        # Filter for Barry Bonds in year 2000
        {'$match': {'playerID': 'bondsba01', 'yearID': 2000}},

        # Group all his records and sum the errors
        {'$group': {'_id': None,
                    'errors': {'$sum': '$E'}}}
    ]))

  return res[0]['errors']


def q13():
  # Get all playerIDs who were allstars in 2000
  allstars = db['AllstarFull'].distinct('playerID', {'yearID': 2000})

  res = list(db['Salaries'].aggregate([
      # Filter for year 2000 and players in the allstars
      {'$match': {'yearID': 2000,
                  'playerID': {'$in': allstars}}},

      # Calculate average salary for the filtered players
      {'$group': {'_id': None,
                  'avg_salary': {'$avg': '$salary'}}}
  ]))

  return res[0]['avg_salary']

def q14():
  # Get all playerIDs who were allstars in 2000
  allstars = db['AllstarFull'].distinct('playerID', {'yearID': 2000})

  res = list(db['Salaries'].aggregate([
      # Filter for year 2000 and players not in the allstars
      {'$match': {'yearID': 2000,
                  'playerID': {'$nin': allstars}}},

      # Calculate average salary for the filtered players
      {'$group': {'_id': None,
                  'avg_salary': {'$avg': '$salary'}}}
  ]))

  return res[0]['avg_salary']



In [43]:

def test_q1():
    result_q1 = q1()
    assert result_q1 == 4834

def test_q2():
    result_q2 = q2()
    assert result_q2 == 73

def test_q3():
    result_q3 = q3()
    assert result_q3 == 'bondsba01'

def test_q4():
    result_q4 = q4()
    assert result_q4 == 7

def test_q5():
    result_q5 = q5()
    assert math.isclose(result_q5, 0.29830576286005, rel_tol=1e-6)

def test_q6():
    result_q6 = q6()
    assert result_q6[0] == 'MIL'
    assert result_q6[1] == 1366

def test_q7():
    result_q7 = q7()
    assert result_q7[0] == 'SEA'
    assert math.isclose(result_q7[1], 0.280799158489005, rel_tol=1e-6)

def test_q8():
    result_q8 = q8()
    assert result_q8 == 2

def test_q9():
    result_q9 = q9()
    assert result_q9 in [1961, 1962, 1966]

def test_q10():
    result_q10 = q10()
    assert int(result_q10) == 1992984

def test_q11():
    result_q11 = q11()
    assert result_q11 == 3

def test_q12():
    result_q12 = q12()
    assert result_q12 == 6

def test_q13():
    result_q13 = q13()
    assert int(result_q13) == 5388841

def test_q14():
    result_q14 = q14()
    assert int(result_q14) == 1692309

In [44]:
test_q1()
print('Q1 - succesful')
test_q2()
print('Q2 - succesful')
test_q3()
print('Q3 - succesful')
test_q4()
print('Q4 - succesful')
test_q5()
print('Q5 - succesful')
test_q6()
print('Q6 - succesful')
test_q7()
print('Q7 - succesful')
test_q8()
print('Q8 - succesful')
test_q9()
print('Q9 - succesful')
test_q10()
print('Q10 - succesful')
test_q11()
print('Q11 - succesful')
test_q12()
print('Q12 - succesful')
test_q13()
print('Q13 - succesful')
test_q14()
print('Q14 - succesful')

print('All asserts are successful!')

Q1 - succesful
Q2 - succesful
Q3 - succesful
Q4 - succesful
Q5 - succesful
Q6 - succesful
Q7 - succesful
Q8 - succesful
Q9 - succesful
Q10 - succesful
Q11 - succesful
Q12 - succesful
Q13 - succesful
Q14 - succesful
All asserts are successful!


In [ ]:
# AI was used to
# 1) understand return types of aggregations function and MongoDB queries in general and fit them to the testing standards
# 2) How to join data from 2 different tables